In [93]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
import util
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

import notebooks.util

%matplotlib

plt.close('all')

%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/iot_telesat_post_processed.h5"

row_limit = -1

hdf_store = pd.HDFStore(file_path)

instances_df = hdf_store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_df = instances_df.sort_index(0)

In [95]:
plt.close('Performance')

# Plot the optimizations
fig = plt.figure(num='Performance', figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

case, target = pickle.load(open('raw/iot_settings.pkl', 'rb'))

N_orbits = case['T_sim_s'] / case['T_orbit_s']
scale_factors = np.array([1/-1e9, 1, 1/1e3, 1]) * \
                np.array([1/N_orbits, 1, 1/N_orbits, 1/case['T_sim_s']]) * \
                np.array([1/8, 1, 1, 100])

# problem, setting, res = pickle.load(open('raw/iot_telesat_nsga3_200_50.pkl', 'rb'))
# util.plot_performance_iot(axs, problem, setting, res, case, target, scale_factors)
# axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='NSGA-3')

problem, setting, res = pickle.load(open('raw/iot_telesat_nsga3_300_75.pkl', 'rb'))
util.plot_performance_iot(axs, problem, setting, res, case, target, scale_factors)
axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='NSGA-3')

problem, setting, res = pickle.load(open('raw/iot_telesat_rnsga2_1e-2_cont_300_75.pkl', 'rb'))
util.plot_performance_iot(axs, problem, setting, res, case, target, scale_factors)
axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='R-NSGA-3')

No handles with labels found to put in legend.


In [96]:
points = []

# Targets
p = np.array([case['T_bitorbit_target'], case['L_sorbit_target'], case['E_Jorbit_target'], case['P_sorbit_target']/3])
p = p * np.array([-1, 1, 1, 1]) * np.array([N_orbits, 1, N_orbits, N_orbits])
p_fmt = p * scale_factors
points.append({'ind': -1, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'r', 'marker': 'x', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Closest to target
problem, setting, res = pickle.load(open('raw/iot_telesat_rnsga2_1e-2_cont_300_75.pkl', 'rb'))
ref = np.array([60, case['E_Jorbit_target'] * N_orbits])
diff= np.abs(res.F[:, [0, 1]] - ref) / ref
ind_close = np.argmin(np.linalg.norm(diff, axis=1))
p = np.array(util.recompute_obj(problem, res, scale_factors=1))
p = p[:, ind_close]
p_fmt = p * scale_factors
points.append({'ind': ind_close, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

fig = plt.figure(num="Performance")
util.plot_points_iot(axs, points, scale_factors=scale_factors)

In [97]:
plt.close('Settings')

fig = plt.figure(num='Settings', figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

problem, setting, res = pickle.load(open('raw/iot_telesat_nsga3_200_50.pkl', 'rb'))
problem, setting, res = pickle.load(open('raw/iot_telesat_rnsga2_1e-2_cont_300_75.pkl', 'rb'))
util.plot_settings(axs, problem, setting, res, scale_factors=scale_factors)

In [98]:
x_pass, x_Ptx_dBm, x_Gtx_dBi, x_B_Hz = util.get_selection(problem, res)

pass_ind = np.nonzero(x_pass[ind_close,:])[0]

T_orbit = case['T_orbit_s']
T_sim = case['T_sim_s']

fig = plt.figure(figsize=(3.2, 2.4))

# Plot all passes
for i, pass_df in instances_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof.values
    d = pass_df.d.values/1000
    line, = plt.plot(tof, d, linewidth=0.1, color='tab:grey')
    #plt.text(tof[0], d[0], '%d' % p)

# Plot used passes
for i, pass_df in instances_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof
    d = pass_df.d/1000
    if p in pass_ind:
        line, = plt.plot(tof, d, linewidth=0.5, color='tab:red')


plt.xlabel('Time of flight [s]')
plt.ylabel('Range [km]')
plt.xlim((0, T_sim))

plt.grid()
PlotUtil.apply_report_formatting()
fig.set_size_inches(3.2*2, 2.4, forward=True)

plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/iot_telesat_range_selection.pdf')
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/iot_telesat_range_selection_%s_%d.svg' % (setting['label'], res.algorithm.n_gen))

In [99]:
hdf_store.close()